In [1]:
%pylab inline
import logging as log
log.getLogger().setLevel(log.INFO)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob

geom_f = "/data/SciBigData/radargrams/geom/"
pattern = geom_f + "/**/*_geom.lbl"
log.info(pattern)

allgeoms = glob.glob(pattern)

log.info(f"found {len(allgeoms)} radargrams geom-files")

INFO:root:/data/SciBigData/radargrams/geom//**/*_geom.lbl
INFO:root:found 21871 radargrams geom-files


In [3]:
!gdalinfo "{allgeoms[0]}"

ERROR 1: CORE_ITEM_BYTES of 0 is not supported in this gdal PDS reader.
gdalinfo failed - unable to open '/data/SciBigData/radargrams/geom/s_0239xx/s_02391001_geom.lbl'.


In [4]:
from datetime import  datetime


In [5]:
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
import regex

from datetime import datetime

In [6]:
import pandas as pd
tab = allgeoms[0][:-3] + "tab"
log.info(tab)
names = ["RADARGRAM COLUMN", "TIME", "LATITUDE", "LONGITUDE", "MARS RADIUS", "SPACECRAFT RADIUS", 
             "RADIAL VELOCITY", "TANGENTIAL VELOCITY", "SZA", "PHASE/1.0E16"]

types = [int, datetime, float, float, float, float, float, float, float, float]
names = [n.replace(" ", "_").replace("/", "_").lower() for n in names]


dtype = [t for t in zip(names, types)]
def lbl_to_geom(lbl_file):
    return lbl_file[:-3] + "tab"

def read_geom_lbl(file):
    o = np.genfromtxt(file, dtype=dtype, delimiter=",")
    table = pd.DataFrame(o)
    table = table[["latitude", "longitude"]]
#     table = pd.read_csv(file, skiprows=0, names=names)
    return table

def pandas_to_geopandas_points(df, xname="longitude", yname="latitude", crs=104971):
    geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
    crs = {'init': f'epsg:{crs}'} #http://www.spatialreference.org/ref/epsg/2263/
    geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)
    return geo_df

def match_id_from_geom_filename(fname):
    m = regex.match(".*_([0-9]*)_geom.lbl", fname)
    return np.int(m.group(1))

def do_all(fname):
    table = read_geom_lbl(lbl_to_geom(fname))
    gdf = pandas_to_geopandas_points(table)
    tid = match_id_from_geom_filename(fname)
    gdf["gid"] = tid
    
    
    
    return gdf

def read_all(geoms):    
    out = []
    for f in geoms:
        out.append(read_geom_lbl(lbl_to_geom(f)))
        
    return out

INFO:root:/data/SciBigData/radargrams/geom/s_0239xx/s_02391001_geom.tab


In [7]:
from ipypb import track

In [ ]:
out = {}
for fname in track(allgeoms):
    table = read_geom_lbl(lbl_to_geom(fname))
#     gdf = pandas_to_geopandas_points(table)
    tid = match_id_from_geom_filename(fname)
    table["gid"] = tid
    table.to_hdf("data.hdf", "sharad", append=True)
    

 [██████████████████████████##################################] 9669/21871 [13:44<00:00, 0.09s/it]

In [ ]:
np.save("data", out)

In [6]:
from multiprocessing import Pool

p = Pool(1)


Exception ignored in: <function BaseGeometry.__del__ at 0x7fb55e110268>
Traceback (most recent call last):
Exception ignored in: <function BaseGeometry.__del__ at 0x7fb55e110268>
  File "/usr/lib/python3.7/site-packages/shapely/geometry/base.py", line 234, in __del__
    self.empty(val=None)
  File "/usr/lib/python3.7/site-packages/shapely/geometry/base.py", line 231, in empty
    self.__geom__ = val
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.7/site-packages/shapely/geometry/base.py", line 234, in __del__
    self.empty(val=None)
Process ForkPoolWorker-1:
  File "/usr/lib/python3.7/site-packages/shapely/geometry/base.py", line 225, in empty
Traceback (most recent call last):
    if not self._is_empty and not self._other_owned and self.__geom__:
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
KeyboardInterrupt
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 358, in put
    obj = _ForkingPic

In [ ]:
out = p.map(do_all, allgeoms)


KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/home/luca/.local/lib/python3.7/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 661, in _abort_queue
    poller.poll(50)
  File "/usr/lib

In [ ]:
final = pd.concat(out)

In [8]:
out = {}
for lbl in allgeoms:
    table = read_geom_lbl(lbl_to_geom(allgeoms[0]))
    gdf = pandas_to_geopandas_points(table)
    tname = "sharad_" + str(match_id_from_geom_filename(lbl))
    out[tname] = gdf



In [9]:
for k , df in out.items():
    i = k[7:]
    df["orbit"] = i



In [10]:
final = pd.concat(out)

In [ ]:
np.save("alldata", final)

In [24]:
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd

# Creating SQLAlchemy's engine to use
engine = create_engine('postgresql://luca@localhost/sharad')


geodataframe = dfg
#... [do something with the geodataframe]

geodataframe['geom'] = geodataframe['geometry'].apply(lambda x: WKTElement(x.wkt, srid=crs))

#drop the geometry column as it is now duplicative
geodataframe.drop('geometry', 1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
geodataframe.to_sql(sharad_orbit_points, engine, if_exists='append', index=False, 
                         dtype={'geom': Geometry('POINT', srid=crs)})



KeyError: 'geometry'

In [24]:
import os
import sqlite3


DB_PATH = os.path.join(os.getcwd(), 'your-database.db')
# Drop all geospatial data
with sqlite3.connect(DB_PATH) as conn:
    for lbl in allgeoms[:10]:
        table = read_geom_lbl(lbl_to_geom(allgeoms[0]))
        gdf = pandas_to_geopandas_points(table)
#         df = gdf.drop(['geometry'], axis=1)
        tname = "sharad_" + str(match_id_from_geom_filename(lbl))
        df.to_sql(tname, conn, if_exists='replace', index=False)
        
        conn.enable_load_extension(True)
        conn.load_extension("mod_spatialite")
        conn.execute("SELECT InitSpatialMetaData(1);")
        conn.execute(            
            f"SELECT AddGeometryColumn({tname}, 'wkb_geometry', {str(crs_id)}, 'POINT', 2);"
        )
        
        
        
        

OperationalError: no such column: sharad_2391001

In [5]:
import fiona
fiona.supported_drivers

DEBUG:fiona.env:GDAL data files are available at built-in paths
DEBUG:fiona.env:PROJ data files are available at built-in paths
DEBUG:fiona.env:Entering env context: <fiona.env.Env object at 0x7f5efeed4550>
DEBUG:fiona.env:Starting outermost env
DEBUG:fiona.env:No GDAL environment exists
DEBUG:fiona.env:New GDAL environment <fiona._env.GDALEnv object at 0x7f5efeec69b0> created
DEBUG:fiona._env:Logging error handler pushed.
DEBUG:fiona._env:All drivers registered.
DEBUG:fiona._env:GDAL data files are available at built-in paths
DEBUG:fiona._env:PROJ data files are available at built-in paths
DEBUG:fiona._env:Started GDALEnv <fiona._env.GDALEnv object at 0x7f5efeec69b0>.
DEBUG:fiona.env:Updated existing <fiona._env.GDALEnv object at 0x7f5efeec69b0> with options {}
DEBUG:fiona.env:Entered env context: <fiona.env.Env object at 0x7f5efeed4550>
DEBUG:fiona.env:Exiting env context: <fiona.env.Env object at 0x7f5efeed4550>
DEBUG:fiona.env:Cleared existing <fiona._env.GDALEnv object at 0x7f5efe

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'raw',
 'DXF': 'raw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'GeoJSON': 'rw',
 'GPKG': 'rw',
 'GML': 'raw',
 'GPX': 'raw',
 'GPSTrackMaker': 'raw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r',
 'TopoJSON': 'r'}

In [58]:
import fiona
fiona.open(allgeoms[0])

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

DriverError: unsupported driver: 'OGR_PDS'

In [54]:
# !man ogr2ogr
# !cat {allgeoms[0]}

In [55]:
!ogr2ogr -a_srs mars2000.wkt -t_srs mars2000.wkt -append -f "ESRI Shapefile" -oo X_POSSIBLE_NAMES=LON* -oo Y_POSSIBLE_NAMES=LAT* ./test.shp {allgeoms[0]}

Warning 6: Normalized/laundered field name: 'RADARGRAM_COLUMN' to 'RADARGRAM_'
Warning 6: Normalized/laundered field name: 'MARS_RADIUS' to 'MARS_RADIU'
Warning 6: Normalized/laundered field name: 'SPACECRAFT_RADIUS' to 'SPACECRAFT'
Warning 6: Normalized/laundered field name: 'RADIAL_VELOCITY' to 'RADIAL_VEL'
Warning 6: Normalized/laundered field name: 'TANGENTIAL_VELOCITY' to 'TANGENTIAL'
Warning 6: Normalized/laundered field name: 'PHASE/1.0E16' to 'PHASE/1.0E'


In [42]:
from ply import lex, yacc

tokens = ['KEYWORD', 'POINTER', 'STRING', 'INT', 'REAL',
          'UNIT', 'DATE', 'END']

literals = list('=(),')

t_POINTER = r'\^[A-Z0-9_]+'
t_ignore_COMMENT = r'/\*.+?\*/'
t_ignore = ' \t\r\n'

# lower case PDS3 to astropy unit translation
unit_translate = dict(v='V', k='K')

def t_KEYWORD(t):
    r'[A-Z][A-Z0-9_:]+'
    if t.value == 'END':
        t.type = 'END'
    return t

def t_DATE(t):
    r'\d\d\d\d-\d\d-\d\d(T\d\d:\d\d(:\d\d(.\d+)?)?)?'
    from astropy.time import Time
    t.value = Time(t.value, scale='utc')
    return t

def t_UNIT(t):
    r'<[\w*^\-/]+>'
    import astropy.units as u

    # most astropy units are lower-case versions of the PDS3 units
    unit = t.value[1:-1].lower()

    # but not all
    if unit in unit_translate:
        unit = unit_translate[unit]

    t.value = u.Unit(unit)
    return t

def t_STRING(t):
    r'"[^"]+"'
    t.value = t.value[1:-1].replace('\r', '')
    return t
 
def t_REAL(t):
    r'[+-]?[0-9]+\.[0-9]*([Ee][+-]?[0-9]+)?'
    t.value = float(t.value)
    return t

def t_INT(t):
    r'[+-]?[0-9]+'
    t.value = int(t.value)
    return t

def t_error(t):
    raise IllegalCharacter(t.value[0])

def p_label(p):
    """label : record
             | label record
             | label END"""
    if len(p) == 2:
        # record
        p[0] = [p[1]]
    elif p[2] == 'END':
        # label END
        p[0] = p[1]
    else:
        # label record
        p[0] = p[1] + [p[2]]

def p_record(p):
    """record : KEYWORD '=' value
              | POINTER '=' STRING
              | POINTER '=' '(' STRING ',' INT ')'"""
    if len(p) == 4:
        p[0] = (p[1], p[3])
    else:
        p[0] = (p[1], (p[4], p[6]))

def p_value(p):
    """value : STRING
             | DATE
             | KEYWORD
             | number
             | quantity
             | sequence"""
    p[0] = p[1]

def p_value_quantity(p):
    """quantity : number UNIT"""
    p[0] = p[1] * p[2]

def p_number(p):
    """number : INT
              | REAL"""
    p[0] = p[1]

def p_sequence(p):
    """sequence : '(' value ')'
                | '(' sequence_values ')'"""
    p[0] = p[2]

def p_sequence_values(p):
    """sequence_values : value ','
                       | sequence_values value ','
                       | sequence_values value"""
    if p[2] == ',':
        p[0] = [p[1]]
    else:
        p[0] = p[1] + [p[2]]

def p_error(p):
    if p:
        print("Syntax error at '%s'" % p.value)
    else:
        print("Syntax error at EOF")

# build the parser
lexer = lex.lex()
parser = yacc.yacc()

# parse the label
with open(allgeoms[0], 'r') as inf:
    records = parser.parse(inf.read(-1))

print (records)

TypeError: <module '__main__'> is a built-in module